#### This is the notebook that runs with open Colab, backend is GPU

In [1]:
!pip install -q graspy

     |████████████████████████████████| 81kB 5.3MB/s 


In [0]:
import warnings
import numpy as np


class SinkhornKnopp:
    def __init__(self, max_iter=1000, epsilon=1e-3):
        if type(max_iter) is int or type(max_iter) is float:
            if max_iter > 0:
                self._max_iter = int(max_iter)
            else:
                msg = "max_iter must be greater than 0"
                raise ValueError(msg)
        else:
            msg = "max_iter is not of type int or float"
            raise TypeError(msg)

        if type(epsilon) is int or type(epsilon) is float:
            if epsilon > 0 and epsilon < 1:
                self._epsilon = int(epsilon)
            else:
                msg = "epsilon must be between 0 and 1 exclusively"
                raise ValueError(msg)
        else:
            msg = "epsilon is not of type int or float"
            raise TypeError(msg)

        self._stopping_condition = None
        self._iterations = 0
        self._D1 = np.ones(1)
        self._D2 = np.ones(1)

    def fit(self, P):
        """
        Fit the diagonal matrices in Sinkhorn Knopp's algorithm
        Parameters
        ----------
        P : 2d array-like
            Must be a square non-negative 2d array-like object, that
            is convertible to a numpy array. The matrix must not be
            equal to 0 and it must have total support for the algorithm
            to converge.
        Returns
        -------
        P_eps : A double stochastic matrix.
        """
        P = np.asarray(P)
        assert np.all(P >= 0)
        assert P.ndim == 2
        assert P.shape[0] == P.shape[1]

        N = P.shape[0]
        max_thresh = 1 + self._epsilon
        min_thresh = 1 - self._epsilon

        # Initialize r and c, the diagonals of D1 and D2
        # and warn if the matrix does not have support.
        r = np.ones((N, 1))
        pdotr = P.T.dot(r)
        total_support_warning_str = (
            "Matrix P must have total support. " "See documentation"
        )
        if not np.all(pdotr != 0):
            warnings.warn(total_support_warning_str, UserWarning)

        c = 1 / pdotr
        pdotc = P.dot(c)
        if not np.all(pdotc != 0):
            warnings.warn(total_support_warning_str, UserWarning)

        r = 1 / pdotc
        del pdotr, pdotc

        P_eps = np.copy(P)
        while (
            np.any(np.sum(P_eps, axis=1) < min_thresh)
            or np.any(np.sum(P_eps, axis=1) > max_thresh)
            or np.any(np.sum(P_eps, axis=0) < min_thresh)
            or np.any(np.sum(P_eps, axis=0) > max_thresh)
        ):

            c = 1 / P.T.dot(r)
            r = 1 / P.dot(c)

            self._D1 = np.diag(np.squeeze(r))
            self._D2 = np.diag(np.squeeze(c))
            P_eps = self._D1.dot(P).dot(self._D2)

            self._iterations += 1

            if self._iterations >= self._max_iter:
                self._stopping_condition = "max_iter"
                break

        if not self._stopping_condition:
            self._stopping_condition = "epsilon"

        self._D1 = np.diag(np.squeeze(r))
        self._D2 = np.diag(np.squeeze(c))
        P_eps = self._D1.dot(P).dot(self._D2)

        return P_eps

In [3]:
import numpy as np
import math
import graspy
from scipy.optimize import linear_sum_assignment
from scipy.optimize import minimize_scalar
from sklearn.utils import check_array
#from skp import SinkhornKnopp
from joblib import Parallel, delayed
from graspy.simulations import er_np
import time
import datetime as dt

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
n = 500
p = 0.3
np.random.seed(1)
G1 = er_np(n=n, p=p)

In [0]:
start = dt.datetime.now()
print ('running time', dt.datetime.now() - start)

running time 0:00:00.000061


# from Ali's codes:

In [0]:
class FastApproximateQAP:
    def __init__(
        self,
        n_init=2,
        init_method="barycenter",
        max_iter=30,
        shuffle_input=True,
        eps=0.1,
        gmp=False,
    ):

        if n_init > 0 and type(n_init) is int:
            self.n_init = n_init
        else:
            msg = '"n_init" must be a positive integer'
            raise TypeError(msg)
        if init_method == "rand":
            self.init_method = "rand"
        elif init_method == "barycenter":
            self.init_method = "barycenter"
            self.n_init = 1
        else:
            msg = 'Invalid "init_method" parameter string'
            raise ValueError(msg)
        if max_iter > 0 and type(max_iter) is int:
            self.max_iter = max_iter
        else:
            msg = '"max_iter" must be a positive integer'
            raise TypeError(msg)
        if type(shuffle_input) is bool:
            self.shuffle_input = shuffle_input
        else:
            msg = '"shuffle_input" must be a boolean'
            raise TypeError(msg)
        if eps > 0 and type(eps) is float:
            self.eps = eps
        else:
            msg = '"eps" must be a positive float'
            raise TypeError(msg)
        if type(gmp) is bool:
            self.gmp = gmp
        else:
            msg = '"gmp" must be a boolean'
            raise TypeError(msg)

    def fit(self, A, B):
        A = check_array(A, copy=True, ensure_2d=True)
        B = check_array(B, copy=True, ensure_2d=True)

        if A.shape[0] != B.shape[0]:
            msg = "Matrix entries must be of equal size"
            raise ValueError(msg)
        elif A.shape[0] != A.shape[1] or B.shape[0] != B.shape[1]:
            msg = "Matrix entries must be square"
            raise ValueError(msg)
        elif (A >= 0).all() == False or (B >= 0).all() == False:
            msg = "Matrix entries must be greater than or equal to zero"
            raise ValueError(msg)

        n = A.shape[0]  # number of vertices in graphs

        if self.shuffle_input:
            node_shuffle_input = np.random.permutation(n)
            A = A[np.ix_(node_shuffle_input, node_shuffle_input)]
            # shuffle_input to avoid results from inputs that were already matched

        obj_func_scalar = 1
        if self.gmp:
            obj_func_scalar = -1

        At = np.transpose(A)  # A transpose
        Bt = np.transpose(B)  # B transpose
        score = math.inf
        perm_inds = np.zeros(n)

        for i in range(self.n_init):

            # setting initialization matrix
            if self.init_method == "rand":
                sk = SinkhornKnopp()
                K = np.random.rand(
                    n, n
                )  # generate a nxn matrix where each entry is a random integer [0,1]
                for i in range(10):  # perform 10 iterations of Sinkhorn balancing
                    K = sk.fit(K)
                J = np.ones((n, n)) / float(
                    n
                )  # initialize J, a doubly stochastic barycenter
                P = (K + J) / 2
            elif self.init_method == "barycenter":
                P = np.ones((n, n)) / float(n)

            grad_P = math.inf  # gradient of P
            n_iter = 0  # number of FW iterations

            # OPTIMIZATION WHILE LOOP BEGINS
            while grad_P > self.eps and n_iter < self.max_iter:

                delta_f = (
                    A @ P @ Bt + At @ P @ B
                )  # computing the gradient of f(P) = -tr(APB^tP^t)
                rows, cols = linear_sum_assignment(
                    obj_func_scalar * delta_f
                )  # run hungarian algorithm on gradient(f(P))
                Q = np.zeros((n, n))
                Q[rows, cols] = 1  # initialize search direction matrix Q

                def f(x):  # computing the original optimization function
                    return obj_func_scalar * np.trace(
                        At
                        @ (x * P + (1 - x) * Q)
                        @ B
                        @ np.transpose(x * P + (1 - x) * Q)
                    )

                alpha = minimize_scalar(
                    f, bounds=(0, 1), method="bounded"
                ).x  # computing the step size                

                P_1 = alpha * P + (1 - alpha) * Q  # Update P
                grad_P = np.linalg.norm(P - P_1)
                P = P_1
                n_iter += 1
            # end of FW optimization loop

            _, perm_inds_new = linear_sum_assignment(
                -P
            )  # Project onto the set of permutation matrices

            score_new = np.trace(
                np.transpose(A) @ B[np.ix_(perm_inds_new, perm_inds_new)]
            )  # computing objective function value

            if score_new < score:  # minimizing
                score = score_new
                if self.shuffle_input:
                    perm_inds = np.array([0] * n)
                    perm_inds[node_shuffle_input] = perm_inds_new
                else:
                    perm_inds = perm_inds_new

        if self.shuffle_input:
            node_unshuffle_input = np.array(range(n))
            node_unshuffle_input[node_shuffle_input] = np.array(range(n))
            A = A[np.ix_(node_unshuffle_input, node_unshuffle_input)]
            score = np.trace(np.transpose(A) @ B[np.ix_(perm_inds, perm_inds)])

        self.perm_inds_ = perm_inds  # permutation indices
        self.score_ = score  # objective function value
        return self

    def fit_predict(self, A, B):
        self.fit(A, B)
        return self.perm_inds_

In [0]:
#from faq import FastApproximateQAP

np.random.seed(1)
G1 = er_np(n=n, p=p)

node_shuffle_input = np.random.permutation(n)
G2 = G1[np.ix_(node_shuffle_input, node_shuffle_input)]

start = dt.datetime.now()
gmp = FastApproximateQAP(n_init=10,init_method="rand",gmp=True)
gmp = gmp.fit(G1,G2)
G2 = G2[np.ix_(gmp.perm_inds_, gmp.perm_inds_)]
print ('running time', dt.datetime.now() - start)

print("Number of edge disagreements: ", sum(sum(abs(G1-G2))))

running time 0:01:08.789331
Number of edge disagreements:  87272.0


In [9]:
#from faq import FastApproximateQAP

np.random.seed(1)
G1 = er_np(n=n, p=p)

node_shuffle_input = np.random.permutation(n)
G2 = G1[np.ix_(node_shuffle_input, node_shuffle_input)]

start = dt.datetime.now()
gmp = FastApproximateQAP(n_init=80,init_method="rand",gmp=True)
gmp = gmp.fit(G1,G2)
G2 = G2[np.ix_(gmp.perm_inds_, gmp.perm_inds_)]
print ('running time', dt.datetime.now() - start)

print("Number of edge disagreements: ", sum(sum(abs(G1-G2))))

running time 0:34:55.938386
Number of edge disagreements:  87324.0


# after adding parallel:

In [0]:
class FastApproximateQAP:
    def __init__(
        self,
        n_init=10,
        init_method="rand",
        max_iter=30,
        shuffle_input=True,
        eps=0.1,
        gmp=False,
    ):

        if n_init > 0 and type(n_init) is int:
            self.n_init = n_init
        else:
            msg = '"n_init" must be a positive integer'
            raise TypeError(msg)
        if init_method == "rand":
            self.init_method = "rand"
        elif init_method == "barycenter":
            self.init_method = "barycenter"
            self.n_init = 1
        else:
            msg = 'Invalid "init_method" parameter string'
            raise ValueError(msg)
        if max_iter > 0 and type(max_iter) is int:
            self.max_iter = max_iter
        else:
            msg = '"max_iter" must be a positive integer'
            raise TypeError(msg)
        if type(shuffle_input) is bool:
            self.shuffle_input = shuffle_input
        else:
            msg = '"shuffle_input" must be a boolean'
            raise TypeError(msg)
        if eps > 0 and type(eps) is float:
            self.eps = eps
        else:
            msg = '"eps" must be a positive float'
            raise TypeError(msg)
        if type(gmp) is bool:
            self.gmp = gmp
        else:
            msg = '"gmp" must be a boolean'
            raise TypeError(msg)

    def fit(self, A, B):
        A = check_array(A, copy=True, ensure_2d=True)
        B = check_array(B, copy=True, ensure_2d=True)

        if A.shape[0] != B.shape[0]:
            msg = "Matrix entries must be of equal size"
            raise ValueError(msg)
        elif A.shape[0] != A.shape[1] or B.shape[0] != B.shape[1]:
            msg = "Matrix entries must be square"
            raise ValueError(msg)
        elif (A >= 0).all() == False or (B >= 0).all() == False:
            msg = "Matrix entries must be greater than or equal to zero"
            raise ValueError(msg)

        n = A.shape[0]  # number of vertices in graphs

        if self.shuffle_input:
            node_shuffle_input = np.random.permutation(n)
            A = A[np.ix_(node_shuffle_input, node_shuffle_input)]
            # shuffle_input to avoid results from inputs that were already matched

        obj_func_scalar = 1
        if self.gmp:
            obj_func_scalar = -1

        At = np.transpose(A)  # A transpose
        Bt = np.transpose(B)  # B transpose
        score = math.inf
        perm_inds = np.zeros(n)

#        for i in range(self.n_init):
        def forloop(init_num):
            
            # setting initialization matrix
            if self.init_method == "rand":
                sk = SinkhornKnopp()
                K = np.random.rand(
                    n, n
                )  # generate a nxn matrix where each entry is a random integer [0,1]
                for i in range(10):  # perform 10 iterations of Sinkhorn balancing
                    K = sk.fit(K)
                J = np.ones((n, n)) / float(
                    n
                )  # initialize J, a doubly stochastic barycenter
                P = (K + J) / 2
            elif self.init_method == "barycenter":
                P = np.ones((n, n)) / float(n)

            grad_P = math.inf  # gradient of P
            n_iter = 0  # number of FW iterations
            
            # OPTIMIZATION WHILE LOOP BEGINS
            while grad_P > self.eps and n_iter < self.max_iter:

                delta_f = (
                    A @ P @ Bt + At @ P @ B
                )  # computing the gradient of f(P) = -tr(APB^tP^t)
                rows, cols = linear_sum_assignment(
                    obj_func_scalar * delta_f
                )  # run hungarian algorithm on gradient(f(P))
                Q = np.zeros((n, n))
                Q[rows, cols] = 1  # initialize search direction matrix Q

                def f(x):  # computing the original optimization function
                    return obj_func_scalar * np.trace(
                        At
                        @ (x * P + (1 - x) * Q)
                        @ B
                        @ np.transpose(x * P + (1 - x) * Q)
                    )

                alpha = minimize_scalar(
                    f, bounds=(0, 1), method="bounded"
                ).x  # computing the step size
                
                P_1 = alpha * P + (1 - alpha) * Q  # Update P
                grad_P = np.linalg.norm(P - P_1)
                P = P_1
                n_iter += 1
            # end of FW optimization loop

            _, perm_inds_new = linear_sum_assignment(
                -P
            )  # Project onto the set of permutation matrices

            score_new = np.trace(
                np.transpose(A) @ B[np.ix_(perm_inds_new, perm_inds_new)]
            )  # computing objective function value

            return score_new, perm_inds_new
        if self.init_method=='barycenter':
            self.n_init=1
            result=forloop(self.n_init)
        else:
            par = Parallel(n_jobs=5)
            result = par(delayed(forloop)(init_num) for init_num in range(self.n_init))
        result = np.mat(result)
        score_new = np.transpose(result[:,0])
        perm_inds_new = result[:,1].tolist()
        #print(score_new)
        #print(perm_inds_new)

        _, column = score_new.shape# get the matrix of a raw and column
        _positon = np.argmin(score_new)# get the index of max in the a
        _, j = divmod(_positon, column)
        #print(j)
        #print(perm_inds_new[j])
        if score_new.min() < score:  # minimizing
            score = score_new.min()
            if self.shuffle_input:
                perm_inds = np.array([0] * n)
                perm_inds[node_shuffle_input] = perm_inds_new[j]
                #print(perm_inds)
            else:
                perm_inds = perm_inds_new[j]
                #print(perm_inds)

        

        if self.shuffle_input:
            node_unshuffle_input = np.array(range(n))
            node_unshuffle_input[node_shuffle_input] = np.array(range(n))
            A = A[np.ix_(node_unshuffle_input, node_unshuffle_input)]
            score = np.trace(np.transpose(A) @ B[np.ix_(perm_inds, perm_inds)])

        self.perm_inds_ = perm_inds  # permutation indices
        self.score_ = score  # objective function value
        return self

    def fit_predict(self, A, B):
        self.fit(A, B)
        return self.perm_inds_



In [7]:
#from par305 import FastApproximateQAP
#n_jobs=-1

n = 500
p = 0.3
np.random.seed(1)
G1 = er_np(n=n, p=p)

node_shuffle_input = np.random.permutation(n)
G2 = G1[np.ix_(node_shuffle_input, node_shuffle_input)]

start = dt.datetime.now()
gmp = FastApproximateQAP(n_init=10,init_method="rand",gmp=True)
gmp = gmp.fit(G1,G2)
G2 = G2[np.ix_(gmp.perm_inds_, gmp.perm_inds_)]
print ('running time', dt.datetime.now() - start)

print("Number of edge disagreements: ", sum(sum(abs(G1-G2))))

running time 0:04:34.542567
Number of edge disagreements:  87276.0


In [10]:
#from par305 import FastApproximateQAP 
#n_jobs=-1

np.random.seed(1)
G1 = er_np(n=n, p=p)

node_shuffle_input = np.random.permutation(n)
G2 = G1[np.ix_(node_shuffle_input, node_shuffle_input)]

start = dt.datetime.now()
gmp = FastApproximateQAP(n_init=80,init_method="rand",gmp=True)
gmp = gmp.fit(G1,G2)
G2 = G2[np.ix_(gmp.perm_inds_, gmp.perm_inds_)]
print ('running time', dt.datetime.now() - start)

print("Number of edge disagreements: ", sum(sum(abs(G1-G2))))

running time 0:36:25.260441
Number of edge disagreements:  87432.0


In [8]:
# if we set n_jobs=5

np.random.seed(1)
G1 = er_np(n=n, p=p)

node_shuffle_input = np.random.permutation(n)
G2 = G1[np.ix_(node_shuffle_input, node_shuffle_input)]

start = dt.datetime.now()
gmp = FastApproximateQAP(n_init=10,init_method="rand",gmp=True)
gmp = gmp.fit(G1,G2)
G2 = G2[np.ix_(gmp.perm_inds_, gmp.perm_inds_)]
print ('running time', dt.datetime.now() - start)

print("Number of edge disagreements: ", sum(sum(abs(G1-G2))))

running time 0:04:37.610873
Number of edge disagreements:  87352.0
